# PROYECTO FINAL APRENDIZAJE AUTOMÁTICO

## Machine Learning aplicado a la predicción de la media de un jugador en FIFA 19

#### Alumnos: Raúl Blas Ruiz y Amaro Blest Polo

El objetivo del proyecto es el de conseguir realizar una predicción sobre si la media (Overall) de un jugador es superior o menor a un valor de 70 puntos. Para ello se van a usar los métodos de machine learning vistos en la asignatura de AAMD:

- **Regresión Logística**
- **Redes Neuronales**
- **Support Vector Machines (SVM)**

Para la realización del proyecto se va a utilizar un dataset obtenido de la plataforma [Kaggle.com](https://www.kaggle.com/karangadiya/fifa19). Este dataset cuenta con más de 70 datos sobre 18.208 jugadores. Entre los datos podemos encontrar algunos como:

- **Age**
- **Nationality**
- **Overall**
- **Potential**
- **Preferred Foot**
- **Weak Foot**
- **Dribling**
- **Etc.**

Gracias a algunos de estos datos vamos a realizar un estudio usando las técnicas de Machine Learning mencionadas anteriormente, para la predicción de un Overall mayor o menor a 70. Los datos escogidos para el desarrollo son los siguientes:

- **Age**
- **Potential**
- **Preferred Foot**
- **Weak foot**
- **Dribling**
- **ShotPower**

Para ello vamos a realizar, en primaria instancia, una serie de modificaciones al dataset para poder empezar con el desarrollo buscado.

In [1]:
# CELDA GLOBAL DE IMPORTS
import numpy as np
import time
import pandas as pd
from pandas import DataFrame
from pandas.io.parsers import read_csv
import matplotlib.pyplot as plt
import scipy.optimize as opt
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import svm

In [2]:
# Haciendo uso de la librería pandas conseguimos un dataFrame con las columnas deseadas a modo de tabla
project_data = pd.read_table('data.csv', sep=',', usecols=['Name', 'Age', 'Overall', 'Potential', 'Preferred Foot', 'Weak Foot', 'Dribbling', 'ShotPower'])
project_data

,Name,Age,Overall,Potential,Preferred Foot,Weak Foot,Dribbling,ShotPower
0,L. Messi,31,94,94,Left,4.0,97.0,85.0
1,Cristiano Ronaldo,33,94,94,Right,4.0,88.0,95.0
2,Neymar Jr,26,92,93,Right,5.0,96.0,80.0
3,De Gea,27,91,93,Right,3.0,18.0,31.0
4,K. De Bruyne,27,91,92,Right,5.0,86.0,91.0
...,...,...,...,...,...,...,...,...
18202,J. Lundstram,19,47,65,Right,2.0,42.0,43.0
18203,N. Christoffersson,19,47,63,Right,2.0,39.0,41.0
18204,B. Worman,16,47,67,Right,3.0,45.0,45.0
18205,D. Walker-Rice,17,47,66,Right,3.0,51.0,64.0


## Comprobación del dataset

Comprobamos que el dataset está completo en todos los datos que vamos a usar. Si no es así mostramos en que columnas se encuentran los datos de tipo NaN.

In [3]:
print('¿Existen datos NaN en el dataset?')
dataNaN = project_data.isnull().values.any()
print(dataNaN)
if dataNaN:
    print('Columnas que contienen datos NaN')
    print(project_data.isnull().any())

¿Existen datos NaN en el dataset?
True
Columnas que contienen datos NaN
Name              False
Age               False
Overall           False
Potential         False
Preferred Foot     True
Weak Foot          True
Dribbling          True
ShotPower          True
dtype: bool


## Eliminación de datos NaN

Dado que en las columnas de Preferred Foot, Weak Foot, Dribbling y ShotPower hemos encontrado que existen datos NaN, vamos a comprobar cuántos jugadores tienen esos datos incompletos.

In [4]:
total_nan_values = project_data.isnull().sum()
total_nan_values

Name               0
Age                0
Overall            0
Potential          0
Preferred Foot    48
Weak Foot         48
Dribbling         48
ShotPower         48
dtype: int64

Ya que como hemos podido comprobar son solo 48 de 18.207 jugadores los que tienen valores NaN a partir de la columna Preferred Foot hemos decidido descartarlos haciendo uso de la función dropna() de DataFrame.

In [5]:
project_data = project_data.dropna()
# Re-comprobamos que ahora no existen valores NaN en ninguna fila
total_nan_values = project_data.isnull().sum()
total_nan_values

Name              0
Age               0
Overall           0
Potential         0
Preferred Foot    0
Weak Foot         0
Dribbling         0
ShotPower         0
dtype: int64

## Transformación de la columna Preferred Foot a números

Para la comodidad en el tratamiento de los datos, hemos decidido cambiar la columna Preferred Foot, que contiene valores 'Right' o 'Left' siendo ahora 'Right' = 0 y 'Left' = 1.

In [8]:
project_data['Preferred Foot'] = project_data['Preferred Foot'].map({'Right': 0, 'Left': 1}, na_action=None)
project_data

<ipython-input-8-d6661a03093a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  project_data['Preferred Foot'] = project_data['Preferred Foot'].map({'Right': 0, 'Left': 1}, na_action=None)


,Name,Age,Overall,Potential,Preferred Foot,Weak Foot,Dribbling,ShotPower
0,L. Messi,31,94,94,1,4.0,97.0,85.0
1,Cristiano Ronaldo,33,94,94,0,4.0,88.0,95.0
2,Neymar Jr,26,92,93,0,5.0,96.0,80.0
3,De Gea,27,91,93,0,3.0,18.0,31.0
4,K. De Bruyne,27,91,92,0,5.0,86.0,91.0
...,...,...,...,...,...,...,...,...
18202,J. Lundstram,19,47,65,0,2.0,42.0,43.0
18203,N. Christoffersson,19,47,63,0,2.0,39.0,41.0
18204,B. Worman,16,47,67,0,3.0,45.0,45.0
18205,D. Walker-Rice,17,47,66,0,3.0,51.0,64.0


## Transformación de la columna Overall

Por último para poder empezar a trabajar en las técnicas de Machine Learning, vamos a transformar la columna de la media de cada jugador. Si el jugador cuenta con una media superior a 70 su valor de Overall será 1, por el contrario si su media es menor o igual a 70 su valor de Overall será 0.